<a href="https://colab.research.google.com/github/SandzhievAndrey/SandzhievAndrey/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Настройка Airflow

Для начала вам необходимо выполнить ряд команд чтобы настроить окружение для дальнейшей работы, это позволит первое время не заниматься настройкой среды исполнения, а сразу начать писать код и работать с Airflow.

In [ ]:
# Установка Airflow
!pip install apache-airflow==2.1.4
!pip install wtforms==2.3.3

# Инициализация базы данных
!airflow db init

In [ ]:
# Создадим необходимые папки
!mkdir /root/airflow/dags
!touch /root/airflow/dags/dag.py

In [ ]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

In [ ]:
# Создадим пользователя Airflow
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

Поместите в dag.py следующий код.

```python
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago
from airflow.operators.dummy_operator import DummyOperator

dag = DAG('dag',schedule_interval=timedelta(days=1), start_date=days_ago(1))
t1 = DummyOperator(task_id='task_1', dag=dag)
t2 = DummyOperator(task_id='task_2',dag=dag)
t3 = DummyOperator(task_id='task_3',dag=dag)
t4 = DummyOperator(task_id='task_4',dag=dag)
t5 = DummyOperator(task_id='task_5',dag=dag)
t6 = DummyOperator(task_id='task_6',dag=dag)
t7 = DummyOperator(task_id='task_7',dag=dag)
```

In [ ]:
# Запуск шедулера
!airflow scheduler -D

  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/


In [ ]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректной работы веб морды
# в среде Google Colab

!pip install pyngrok
!ngrok authtoken <YOUR TOKEN> # найти его можно https://dashboard.ngrok.com/get-started/setup 

# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/endpoints/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http -log=stdout 18273 > /dev/null &

После запуска команды выше, перейдите по адресу в ngrok и подождите  пока появится DAG с именем dag

### Задачи

In [ ]:
# Используйте данный шаблон для решения
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago
from airflow.operators.python import PythonOperator
from airflow.exceptions import AirflowSkipException, AirflowFailException

dag = DAG('dag',
          schedule_interval=timedelta(days=1), 
          start_date=days_ago(1))


# Функция которая всегда верна
def success():
  pass

# Функция которая скипает задачу
def skip():
  raise AirflowSkipException

# Функция которая падает с ошибкой
def failed():
  raise AirflowFailException

task_0 = PythonOperator(
  task_id='task_0',
  python_callable=success,
  dag=dag
)

task_1 = PythonOperator(
  task_id='task_1',
  python_callable=skip,
  dag=dag
)

task_2 = PythonOperator(
  task_id='task_2',
  python_callable=failed,
  dag=dag
)

task_3 = PythonOperator(
  task_id='task_3',
  trigger_rule=<ВАШ КОД>',
  python_callable=lambda: print("Success"),
  dag=dag
)

[task_0, task_1, task_2] >> task_3



task_4 = PythonOperator(
  task_id='task_4',
  python_callable=success,
  dag=dag
)

task_5 = PythonOperator(
  task_id='task_5',
  python_callable=failed,
  dag=dag
)

task_6 = PythonOperator(
  task_id='task_6',
  python_callable=failed,
  dag=dag
)

task_7 = PythonOperator(
  task_id='task_7',
  python_callable=lambda: print("Success"),
  trigger_rule=<ВАШ КОД>,
  dag=dag
)

[task_4, task_5,task_6] >> task_7

In [ ]:
# Решение на разработку

Даг нужно написать в файл /root/airflow/dags/dag.py. Проверку можно сделать в веб интерфейсе. Прежде чем даг появится, может пройти ~ 2-3 минут.